### Описание задания
Загрузите данные и посчитайте модели линейной регрессии для 50 зданий по ансамблю регрессионных моделей: в первой модели весь оптимальный набор метеорологических данных, во второй - дни недели и праздники, в третьей - недели года, в четвертой - месяцы. Финальное значение показателя рассчитайте как взвешенное арифметическое показателей всех моделей, взяв веса для первой и второй модели как 3/8, а для третьей и четвертой - как 1/8.

Загрузите данные решения, посчитайте значение энергопотребления для требуемых дат для тех зданий, которые посчитаны в модели, и выгрузите результат в виде CSV-файла (submission.csv).

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/test.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz

In [1]:
# Подключение библиотек
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from scipy.interpolate import interp1d
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import os

In [2]:
# Функция оптимизации памяти
def reduce_mem_usage (df):
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == "float":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo("f2").min and c_max < np.finfo("f2").max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo("f4").min and c_max < np.finfo("f4").max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == "int":
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == "timestamp":
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != "datetime":
            df[col] = df[col].astype("category")
    end_mem = df.memory_usage().sum() / 1024**2
    print('Потребление памяти меньше на', round(start_mem - end_mem, 2), 'Мб (минус', round(100 * (start_mem - end_mem) / start_mem, 1), '%)')
    return df


In [3]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
weather = weather[weather["site_id"] == 0]
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")
energy = energy[energy["building_id"] < 50]
energy = pd.merge(left=energy, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings

In [4]:
# Интерполируем недостающие данные по погоде
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')

In [5]:
#Дополним данные по погоде первой и второй производной по температуре воздуха
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]

In [6]:
# Объединяем данные
energy = energy.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
energy = pd.merge(left=energy, right=weather, how="left",
                  left_index=True, right_index=True)
energy.reset_index(inplace=True)
energy = energy.drop(columns=["meter", "site_id", "year_built",
                              "square_feet", "floor_count"], axis=1)
energy = reduce_mem_usage(energy)
del weather
print (energy.info())

Потребление памяти меньше на 30.52 Мб (минус 71.2 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432456 entries, 0 to 432455
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              432456 non-null  datetime64[ns]
 1   building_id            432456 non-null  int8          
 2   meter_reading          432456 non-null  float16       
 3   primary_use            432456 non-null  category      
 4   air_temperature        432456 non-null  float16       
 5   cloud_coverage         432456 non-null  float16       
 6   dew_temperature        432456 non-null  float16       
 7   precip_depth_1_hr      432456 non-null  float16       
 8   sea_level_pressure     432456 non-null  float16       
 9   wind_direction         420144 non-null  float16       
 10  wind_speed             432456 non-null  float16       
 11  air_temperature_diff1  432456 non-null  float16    

In [7]:
# Обогащение данных по дате
energy["hour"] = energy["timestamp"].dt.hour.astype("int8")
energy["weekday"] = energy["timestamp"].dt.weekday.astype("int8")
energy["week"] = energy["timestamp"].dt.isocalendar().week.astype("int8")
energy["month"] = energy["timestamp"].dt.month.astype("int8")
energy["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    energy['is_w' + str(week)] = energy['week'].isin([week]).astype("int8")
for month in range(1,13):
    energy['is_m' + str(month)] = energy['month'].isin([month]).astype("int8")

In [8]:
#Создаем колонку логарифма целевого показателя
energy["meter_reading_log"] = np.log(energy["meter_reading"] + 1)

In [9]:
# Выгружаем данные по energy в файл HDF5
energy.to_hdf('energy.0-50.ready.h5', "energy", format='table', complevel=9, mode="w")
print ("HDF5:", os.path.getsize(os.getcwd() + '\energy.0-50.ready.h5'))

HDF5: 1713788


In [10]:
# Сохраняем данные в файл hdf
energy = pd.read_hdf('energy.0-50.ready.h5', "energy")
print (energy.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 432456 entries, 0 to 432455
Data columns (total 92 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              432456 non-null  datetime64[ns]
 1   building_id            432456 non-null  int8          
 2   meter_reading          432456 non-null  float16       
 3   primary_use            432456 non-null  category      
 4   air_temperature        432456 non-null  float16       
 5   cloud_coverage         432456 non-null  float16       
 6   dew_temperature        432456 non-null  float16       
 7   precip_depth_1_hr      432456 non-null  float16       
 8   sea_level_pressure     432456 non-null  float16       
 9   wind_direction         420144 non-null  float16       
 10  wind_speed             432456 non-null  float16       
 11  air_temperature_diff1  432456 non-null  float16       
 12  air_temperature_diff2  432456 non-null  floa

In [11]:
# Загружаем данные для расчета, объединяем, оптимизируем, обогащаем
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz",
                       usecols=["site_id", "building_id"])

weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_test.csv.gz")
weather = weather[weather["site_id"] == 0]
weather = weather.drop(columns=["wind_direction"], axis=1)
results = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/test.csv.gz")

results = results[(results["building_id"] < 50) & (results["meter"] == 0)]

results = pd.merge(left=results, right=buildings, how="left",
                   left_on="building_id", right_on="building_id")
del buildings
results = results.drop(columns=["meter"], axis=1)
print (results.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 876000 entries, 0 to 875999
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   row_id       876000 non-null  int64 
 1   building_id  876000 non-null  int64 
 2   timestamp    876000 non-null  object
 3   site_id      876000 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.4+ MB
None


In [12]:
weather["precip_depth_1_hr"] = weather["precip_depth_1_hr"].apply(lambda x:x if x>0 else 0)
interpolate_columns = ["air_temperature", "dew_temperature",
                       "cloud_coverage", "wind_speed",
                       "precip_depth_1_hr", "sea_level_pressure"]
for col in interpolate_columns:
    weather[col] = weather[col].interpolate(limit_direction='both',
                            kind='cubic')
weather["air_temperature_diff1"] = weather["air_temperature"].diff()
weather.at[0, "air_temperature_diff1"] = weather.at[1, "air_temperature_diff1"]
weather["air_temperature_diff2"] = weather["air_temperature_diff1"].diff()
weather.at[0, "air_temperature_diff2"] = weather.at[1, "air_temperature_diff2"]

In [13]:
results = results.set_index(["timestamp", "site_id"])
weather = weather.set_index(["timestamp", "site_id"])
results = pd.merge(left=results, right=weather, how="left",
                  left_index=True, right_index=True)
results.reset_index(inplace=True)
results = results.drop(columns=["site_id"], axis=1)
del weather
results = reduce_mem_usage(results)
print (results.info())

Потребление памяти меньше на 49.29 Мб (минус 67.0 %)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              876000 non-null  datetime64[ns]
 1   row_id                 876000 non-null  int32         
 2   building_id            876000 non-null  int8          
 3   air_temperature        876000 non-null  float16       
 4   cloud_coverage         876000 non-null  float16       
 5   dew_temperature        876000 non-null  float16       
 6   precip_depth_1_hr      876000 non-null  float16       
 7   sea_level_pressure     876000 non-null  float16       
 8   wind_speed             876000 non-null  float16       
 9   air_temperature_diff1  876000 non-null  float16       
 10  air_temperature_diff2  876000 non-null  float16       
dtypes: datetime64[ns](1), float16(8), int32(1), int8(1)

In [14]:
results["hour"] = results["timestamp"].dt.hour.astype("int8")
results["weekday"] = results["timestamp"].dt.weekday.astype("int8")
results["week"] = results["timestamp"].dt.isocalendar().week.astype("int8")
results["month"] = results["timestamp"].dt.month.astype("int8")
results["date"] = pd.to_datetime(energy["timestamp"].dt.date)
dates_range = pd.date_range(start='2016-12-31', end='2018-06-01')
us_holidays = calendar().holidays(start=dates_range.min(),
                                  end=dates_range.max())
results['is_holiday'] = results['date'].isin(us_holidays).astype("int8")
for weekday in range(0,7):
    results['is_wday' + str(weekday)] = results['weekday'].isin([weekday]).astype("int8")
for week in range(1,54):
    results['is_w' + str(week)] = results['week'].isin([week]).astype("int8")
for month in range(1,13):
    results['is_m' + str(month)] = results['month'].isin([month]).astype("int8")
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 89 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   timestamp              876000 non-null  datetime64[ns]
 1   row_id                 876000 non-null  int32         
 2   building_id            876000 non-null  int8          
 3   air_temperature        876000 non-null  float16       
 4   cloud_coverage         876000 non-null  float16       
 5   dew_temperature        876000 non-null  float16       
 6   precip_depth_1_hr      876000 non-null  float16       
 7   sea_level_pressure     876000 non-null  float16       
 8   wind_speed             876000 non-null  float16       
 9   air_temperature_diff1  876000 non-null  float16       
 10  air_temperature_diff2  876000 non-null  float16       
 11  hour                   876000 non-null  int8          
 12  weekday                876000 non-null  int8

In [15]:
# Набор параметров для каждой модели
lr_weather_columns = ["meter_reading_log", "hour", "building_id",
                      "air_temperature", "dew_temperature",
                      "sea_level_pressure", "wind_speed",
                      "air_temperature_diff1", "air_temperature_diff2",
                      "cloud_coverage"]

lr_days_columns = ["meter_reading_log", "hour", "building_id", "is_holiday"]
for wday in range(0,7):
    lr_days_columns.append("is_wday" + str(wday))
    
lr_weeks_columns = ["meter_reading_log", "hour", "building_id"]
for week in range(1,54):
    lr_weeks_columns.append("is_w" + str(week))
    
lr_months_columns = ["meter_reading_log", "hour", "building_id"]
for month in range(1,13):
    lr_months_columns.append("is_m" + str(month))

hours = range(0, 24)
buildings = range(0, energy["building_id"].max() + 1)

In [16]:
# Определим функцию для построения модели.
def train_model (df, columns):
    df_train_lr = pd.DataFrame(df, columns=columns)
    df_lr = [[]]*len(buildings)
    for building in buildings:
        df_lr[building] = [[]]*len(hours)
        df_train_b = df_train_lr[df_train_lr["building_id"]==building]
        for hour in hours:
            df_train_bh = df_train_b[df_train_b["hour"]==hour]
            y = df_train_bh["meter_reading_log"]
            x = df_train_bh.drop(labels=["meter_reading_log",
                "hour", "building_id"], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            df_lr[building][hour] = model.coef_
            df_lr[building][hour] = np.append(df_lr[building][hour], model.intercept_)
    return df_lr

In [17]:
# Строим модели по всем наборам данных
weather_lr = train_model (energy, lr_weather_columns)
days_lr = train_model (energy, lr_days_columns)
weeks_lr = train_model (energy, lr_weeks_columns)
months_lr = train_model (energy, lr_months_columns)

In [18]:
# Рассчитываем суммарную модель с учетом весовых коэффициентов
weather_lr = np.asarray(weather_lr, dtype=np.float16)*0.375
days_lr = np.asarray(days_lr, dtype=np.float16)*0.375
weeks_lr = np.asarray(weeks_lr, dtype=np.float16)*0.125
months_lr = np.asarray(months_lr, dtype=np.float16)*0.125

models = [weather_lr, days_lr, weeks_lr, months_lr]
models_col = [lr_weather_columns, lr_days_columns, lr_weeks_columns, lr_months_columns]

def calculate_model (x):
    lr = 0
    for i in range(len(models)):
        model = models[i][x.building_id][x.hour]
        lr_columns = models_col[i]
        if len(model) > 0:
            lr += np.sum([x[col] * model[i] for i,col in enumerate(lr_columns[3:])])
            lr += model[len(lr_columns)-3]
    lr = np.exp(lr)
    if lr < 0 or lr != lr or lr*lr == lr:
        lr = 0
    x["meter_reading"] = lr
    return x

results = results.apply(calculate_model, axis=1, result_type="expand")

In [19]:
results = pd.DataFrame(results, columns=["row_id", "meter_reading"])
print (results.head())
results.info()

   row_id  meter_reading
0       0      19.453964
1       1      13.456142
2       2       3.462408
3       3      23.195927
4       4      53.801033
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 876000 entries, 0 to 875999
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   row_id         876000 non-null  int64  
 1   meter_reading  876000 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 13.4 MB


In [20]:
results.to_csv("submission.csv",index=False)

In [21]:
del energy
del results